<img src="Figs/Banner.png" width="100%" />
<font face="Calibri">
<br>
<font size="7"> <b> GEOS 639 Geodetic Imaging <b> </font>

<font size="5"> <b> Lab 4: InSAR Time Series Analysis using MintPy and ARIA products<font color='rgba(200,0,0,0.2)'>  -- [20 Points] </font></b> </font>
<br>
<font size="4" color='rgba(200,0,0,0.2)'><b>Assignment Due Date: </b> April 05, 2022 </font>
<br> <img src="Figs/NASALogo.png" width="250" align="right" /> <br> 
<font size="4"> <b> Eric Fielding, David Bekaert, Heresh Fattahi and Zhang Yunjun with modifications by Franz J Meyer</b> 
<font size="3">  <br>
<font> <b>Date: </b> March 19, 2022 </font>
</font>

<font face="Calibri" size="3">This notebook is a second modification by Eric Fielding from an earlier version of the notebook (https://nbviewer.jupyter.org/github/aria-tools/ARIA-tools-docs/blob/master/JupyterDocs/NISAR/L2_interseismic/mintpySF/smallbaselineApp_aria.ipynb) by David Bekaert, Heresh Fattahi and Zhang Yunjun that was originally focused on San Francisco. 

 This notebook is a modification from the [original](https://nbviewer.jupyter.org/github/insarlab/MintPy-tutorial/blob/main/smallbaselineApp_aria.ipynb) by Heresh Fattahi and Zhang Yunjun. 


**Mapping landslide motion with InSAR time series**

The Caltech-JPL ARIA project in partnership with NASA Getting Ready for NISAR (GRFN) project has been generated surface displacement products (interferograms) mimicking the NISAR L2 GUNW (Geocoded Unwrapped phase interferograms) product formatting. The interferograms are stored at the NASA ASF DAAC, and are accessible with an Open Source set of tools called ARIA-tools. The Miami Insar Timeseries software in PYthon (MintPy), an open-source package for InSAR time-series analysis, is compatible with the outputs from the ARIA-tools package, and in combination with the ARIA-tools pre-processor can be used to estimate ground displacement time-series. 

The Jupyter notebook presented here is meant as an practical example on the use of Jupyter for exploring landslide displacements. In the example below, we will demonstrate a time-series derived from ARIA standard InSAR products over the Los Angeles, California area revealing landslide motion on the Palos Verdes peninsula. 

See the other tutorials on ARIA-tools to learn more about how to use that package.
</font>

<font face="Calibri" size="3">
<div class="alert alert-success">
<b>To save time, we have pre-ran the ARIA-tools package and data loading into MintPy</b> 

```!ariaDownload.py -b '33.5 34.5 -119.0 -117.9' --track 71```
    
```!ariaTSsetup.py -f 'products/*.nc' -b '33.65 33.9 -118.45 -118.15' --mask Download```
    
```!smallbaselineApp.py -t LASenDT71.txt --dostep load_data```
    

</div>
</font>

<font face="Calibri" size="3">
<div class="alert alert-success">
 
<b>The staged data was uploaded to an S3 data bucket in AWS so that they can be easily downloaded into the OpenSARLab:</b>
    
```!aws --region=us-west-2 --no-sign-request s3 cp s3://asf-jupyter-data-west/Fielding/Stack.zip Stack.zip```

When users are not leveraging openSARlabs, they should start from ARIA-tools as download from S3 will not work.
</div>
</font>

<div class="alert alert-danger">
<font size="4"> <font color='rgba(200,0,0,0.2)'> <b>THIS NOTEBOOK INCLUDES TWO HOMEWORK ASSIGNMENTS.</b></font> 
<br> 
<font size="3"> The homework assignments in this lab are indicated by markdown fields with <font color='rgba(200,0,0,0.2)'><b>red background</b></font>. Please complete these assignments to achieve full score. </font> <br>

<font size="3"> To submit your homework, please download your completed Jupyter Notebook from the server both as PDF (*.pdf) and Notebook file (*.ipynb) and submit them as a ZIP bundle via the GEOS 639 Canvas page. To download, please select the following options in the main menu of the notebook interface:

<ol type="1">
  <li><font color='rgba(200,0,0,0.2)'> <b> Save your notebook with all of its content</b></font> by selecting <i> File / Save and Checkpoint </i> </li>
  <li><font color='rgba(200,0,0,0.2)'> <b>To export in Notebook format</b></font>, click the <i>radio button next to the notebook file in the main Jupyter Hub browser tab. Once clicked, a download field will appear near the top of the page.</i></li>
  <li><font color='rgba(200,0,0,0.2)'> <b>To export in PDF format</b></font>, right-click on your browser window and print the browser content to PDF</li>
</ol>

Contact me at fjmeyer@alaska.edu should you run into any problems.
</font>
</font>
</div>

# 0. Notebook setup

## 0.1. Set Conda Environment

In [ ]:
import url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

notebookUrl = notebookUrl.value
user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/unavco':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "unavco" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select "unavco" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "unavco" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

# 0.2. Import Python Libraries and Set Environment Variables

The two cells below must be run each time the notebook is started to ensure correct set-up of the notebook.

In [ ]:
import os
import zipfile
import numpy as np
import h5py
from osgeo import gdal
from datetime import datetime
import csv
import opensarlab_lib as asfn
import urllib
from pathlib import Path
import re

# verify mintpy install is complete:
try:
    from mintpy.objects.insar_vs_gps import plot_insar_vs_gps_scatter
    import mintpy.tsview
    from mintpy.tsview import timeseriesViewer
    from mintpy.unwrap_error_phase_closure import plot_num_triplet_with_nonzero_integer_ambiguity
    from mintpy import view, tsview, plot_network, plot_transection, plot_coherence_matrix
except ImportError:
    raise ImportError("Looks like mintPy is not fully installed")
    
def write_config_file(out_file, CONFIG_TXT, mode='a'): 
    """Write configuration files for MintPy to process ARIA sample products"""
    if not os.path.isfile(out_file) or mode == 'w':
        with open(out_file, "w") as fid:
            fid.write(CONFIG_TXT)
        print('write configuration to file: {}'.format(out_file))
    else:
        with open(out_file, "a") as fid:
            fid.write("\n" + CONFIG_TXT)
        print('add the following to file: \n{}'.format(CONFIG_TXT))

# visualization functions
def tsview(fname, yx=None, figsize_img=[5, 4], figsize_pts=[5, 2]):
    """Plot input file using tsview.py"""
    cmd = 'tsview.py {} --ms 4 --noverbose'.format(fname)
    if yx is not None:
        cmd += ' --yx {} {}'.format(yx[0], yx[1])
    obj = timeseriesViewer(cmd)
    obj.configure()
    obj.figsize_img = figsize_img
    obj.figsize_pts = figsize_pts
    obj.plot()
    return obj

In [ ]:
# define the work directory
work_dir = f"{os.path.abspath(os.getcwd())}/data_LA"
print("Work directory: ", work_dir)
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)
    print(f'Create directory: {work_dir}')
print(f'Go to work directory: {work_dir}')
os.chdir(work_dir)  
    
if not os.path.isfile('Stack.zip'):
    !aws --region=us-west-2 --no-sign-request s3 cp s3://asf-jupyter-data-west/Fielding/Stack.zip Stack.zip 
      
# verify if download was succesfull
if os.path.isfile('Stack.zip'):
    with zipfile.ZipFile('Stack.zip', 'r') as zip_ref:
        zip_ref.extractall(os.getcwd())
    print('S3 pre-staged data retrieval was successfull')
else:
    print("Download outside openSarLabs is not supported.\nAs alternative please start from ARIA-tools with the commandline calls provided at the top of this notebook")          

<font face="Calibri" size="3">The following command will download all the ARIA standard products over the LA area, which is 575 products at the time of this writing. This will take more than two hours to complete if the data is not already downloaded.</font>

In [ ]:
# download data for descending track 71 over Los Angeles area
#!ariaDownload.py -b '33.5 34.5 -119.0 -117.9' --track 71

<font face="Calibri" size="3">This ARIA time-series setup would cover the whole Los Angeles area and would take a while to process, so we are skipping this setup.</font>

In [ ]:
#!ariaTSsetup.py -f 'products/*.nc' -b '33.5 34.5 -119.0 -117.9' --mask Download

<font face="Calibri" size="3">The following ARIA time-series setup `ariaTSsetup.py` extracts the data that covers only a small area around the Palos Verdes peninsula southwest of Los Angeles to speed the time-series processing, which we specify with the bounding box. We also download the water mask to avoid using data over the ocean.</font>

In [ ]:
#!ariaTSsetup.py -f 'products/*.nc' -b '33.65 33.9 -118.45 -118.15' --mask Download

# 1. smallbaselineApp.py overview

<font face="Calibri" size="3">This application provides a workflow which includes several steps to invert a stack of unwrapped interferograms and apply different corrections to obtain ground displacement timeseries.  
The workflow consists of two main blocks:

* correcting unwrapping errors and inverting for the raw phase time-series (blue ovals),
* correcting for noise from different sources to obtain the displacement time-series (green ovals).

Some steps are optional, which are switched off by default (marked by dashed boundaries). Configuration parameters for each step are initiated with default values in a customizable text file: [smallbaselineApp.cfg](https://github.com/insarlab/MintPy/blob/master/mintpy/defaults/smallbaselineApp.cfg). In this notebook, we will walk through some of these steps, for a complete example see the [MintPy repository](https://github.com/insarlab/MintPy).

<p align="left">
  <img width="600" src="NotebookAddons/MintPyWorkflow.jpg">
</p>     
<p style="text-align: center;">
    (Figure from Yunjun et al., 2019)
</p>
</font>

## 1.1 Processing Steps of smallbaselineApp.py

<font face="Calibri" size="3">The MintPy **smallbaselineApp.py** application provides a workflow to invert a stack of unwrapped interferograms and apply different (often optional) corrections to obtain ground displacement timeseries. A detailed overview of the options can be retrieved by involking the help option:</font>

In [ ]:
!smallbaselineApp.py --help

## 1.2 Configuring Processing Parameters

<font face="Calibri" size="3">The processing parameters for the **smallbaselineApp.py** are controlled through a configuration file. If no file is provided the default [smallbaselineApp.cfg](https://github.com/insarlab/MintPy/blob/master/mintpy/defaults/smallbaselineApp.cfg) configuration is used. Here we use `LASenDT71.txt`, which already constains selected, manually modified configuration parameters for this time-series analysis.</font>

# 2. Small Baseline Time Series Analysis

## 2.1. Loading ARIA data into MintPy

<font face="Calibri" size="3">The [ARIA-tools package](https://github.com/aria-tools/ARIA-tools) is used as a pre-processor for MintPY. It has a download tool that wraps around the ASF DAAC API, and includes tools for stitching/cropping and time-series preparation. The output of the time-series preparation is compatible with the [data directory](https://mintpy.readthedocs.io/en/latest/dir_structure/) structure from MintPy. To save time, we have already pre-ran these steps. The commands used were:

```!ariaDownload.py -b '33.5 34.5 -119.0 -117.9' --track 71```

```!ariaTSsetup.py -f 'products/*.nc' -b '33.65 33.9 -118.45 -118.15' --mask Download```
</font>

<font face="Calibri" size="3">The `ariaTSsetup.py` step above (or the pre-processed Stack.zip) extracted the data for the subset we specified and found a total of 439 products that cover our study area. Now we load the data for the subset area into MintPy.</font>

In [ ]:
# define the MintPy time-series directory
mint_dir = work_dir+'/MintPy'
print("MintPy directory: ", mint_dir)
if not os.path.isdir(mint_dir):
    os.makedirs(mint_dir)
    print('Create directory: {}'.format(mint_dir))

# copy the configuration file
os.chdir(work_dir)  
!cp LASenDT71.txt MintPy

print('Go to work directory: {}'.format(mint_dir))
os.chdir(mint_dir)  

!smallbaselineApp.py LASenDT71.txt --dostep load_data

<font face="Calibri" size="3">The output of the loading step is an "inputs" directory containing two HDF5 files:
- ifgramStack.h5: This file contains 6 dataset cubes (e.g. unwrapped phase, coherence, connected components etc.) and multiple metadata.  
- geometryGeo.h5: This file contains geometrical datasets (e.g., incidence/azimuth angle, masks, etc.). 
</font>

In [ ]:
!ls inputs

<font face="Calibri" size="3">
<div class="alert alert-success">
<b>info.py :</b> 
To get general infomation about a MintPy product, run info.py on the file.   
</div>
</font>

In [ ]:
!info.py inputs/ifgramStack.h5

In [ ]:
!info.py inputs/geometryGeo.h5

## 2.2. Plotting the Interferogram Network

<font face="Calibri" size="3">Running **plot_network.py** gives an overview of the network and the average coherence of the stack. The program creates multiple files as follows:
- ifgramStack_coherence_spatialAvg.txt: Contains interferogram dates, average coherence temporal and spatial baseline separation.
- Network.pdf: Displays the network of interferograms on time-baseline coordinates, colorcoded by average coherence of the interferograms. 
- CoherenceMatrix.pdf shows the average coherence pairs between all available pairs in the stack.
</font>

In [ ]:
!smallbaselineApp.py LASenDT71.txt  --dostep modify_network
plot_network.main(['inputs/ifgramStack.h5'])

## 2.3.  Mask Generation

<font face="Calibri" size="3">Before running the time-series inversion, one may want to looks at average coherence in the stack. To create a map of average spatial coherence use `temporal_average.py`:</font>

In [ ]:
!temporal_average.py ./inputs/ifgramStack.h5 -d coherence -o avgSpatialCoh.h5
view.main('avgSpatialCoh.h5 --noverbose'.split())

<font face="Calibri" size="3"><b>Mask files</b> can be can be used to mask pixels in the time-series processing. Below we generate a mask file based on the connected components, which is a metric for unwrapping quality.</font>

In [ ]:
!generate_mask.py  inputs/ifgramStack.h5  --nonzero  -o maskConnComp.h5  --update
view.main('maskConnComp.h5 -c gray --noverbose'.split())

## 2.4 Reference Point Selection


<font face="Calibri" size="3">The interferometric phase is a relative observation by nature. The phases of each unwrapped interferogram are relative with respect to an arbitrary pixel. Therfore we need to reference all interferograms to a common reference pixel.
The step "reference_point" selects a common reference pixel for the stack of interferograms. The default approach of mintpy is to choose a pixel with highest spatial coherence in the stack. Other options include specifying the longitude and latitude of the desired reference pixel or the line and column number of the refence pixel.</font>

In [ ]:
!smallbaselineApp.py LASenDT71.txt --dostep reference_point

<font face="Calibri" size="3">Running the "reference_step" adds additional attributes "REF_X, REF_Y" and "REF_LON, REF_LAT" to the ifgramStack.h5 file. To see the attributes of the file run info.py:</font>

In [ ]:
!info.py inputs/ifgramStack.h5 | egrep 'REF_'

<font face="Calibri" size="3">In this case, I set the reference point latitude and longitude to be in a location close to the Portuguese Bend Landslide, and MintPy calculated the X and Y locations.</font>

## 2.5 Inverting the Small Baseline Network

<font face="Calibri" size="3">In the next step we invert the network of differential unwrapped interferograms to estimate the time-series of unwrapped phase with respect to a reference acquisition date. By default mintpy selects the first acquisition. The estimated time-series is converted to distance change from radar to target and is provided in meters. </font>

In [ ]:
!smallbaselineApp.py LASenDT71.txt --dostep invert_network

<font face="Calibri" size="3">The timeseries file contains three datasets:
- the "time-series" which is the interferometric range change for each acquisition relative to the reference acquisition,
- the "date" dataset which contains the acquisition date for each acquisition,
- the "bperp" dataset which contains the timeseries of the perpendicular baseline.  
</font>

<font face="Calibri" size="3">The function call below allows you to <b>visualize the inverted InSAR <i>phase</i> time series</b>. The phase is referenced to the first image acquisition. Hence, early images tend to have less phase signal.    
</font>

In [ ]:
view.main('timeseries.h5 -v -5 5 --noaxis'.split())

<div class="alert alert-danger">
<font size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>ASSIGNMENT #1</u>:  </font> Information content of the phase time series:</b> <font color='rgba(200,0,0,0.2)'> -- [8 Points] </font> </font>
You may see that there is a lot of variation in this time series, with phase "colors" changing from blue to red in some areas. In addition to surface displacement signals $\phi_{disp}$, which other phase components may cause this variation in time. Name and briefly explain two likely causes for these phase variations.

</div>

<hr>
<div class="alert alert-danger">
<i><font color='rgba(200,0,0,0.2)'> Question 1.1 [4 Points]:</font></i> 

NAME AND BRIEFLY DESCRIBE <b>FIRST</b> NUISANCE PHASE SIGNAL IN THE ABOVE TIME SERIES by adding text below:
</div>

<hr>
<div class="alert alert-danger">
<i><font color='rgba(200,0,0,0.2)'> Question 1.2 [4 Points]:</font></i> 

NAME AND BRIEFLY DESCRIBE <b>SECOND</b> NUISANCE PHASE SIGNAL IN THE ABOVE TIME SERIES by adding text below:
</div>

## 2.6 Estimating the Long-Term Velocity Rate

<font face="Calibri" size="3">The ground deformation caused by many geophysical or anthropogenic processes are linear at first order approximation. Therefore it is common to estimate the rate of the ground deformation, which is the slope of linear fit to the time-series. </font>

In [ ]:
!smallbaselineApp.py LASenDT71.txt --dostep velocity

In [ ]:
scp_args = 'velocity.h5 velocity -v -1 1'
view.main(scp_args.split())

<font face="Calibri" size="3">
<div class="alert alert-success">
<b>Note :</b> 
Negative values indicates that target is moving away from the radar (i.e., Subsidence in case of vertical deformation).
Positive values indicates that target is moving towards the radar (i.e., uplift in case of vertical deformation). 
    The line of sight (LOS) for this descending Sentinel-1 track is up and east from ground to radar.
</div>
</font>

<font face="Calibri" size="3"><b>Obvious features in the estimated velocity map:</b>

1) There are several features with significant velocity in this area.

2) The negative LOS feature on the Palos Verdes peninsula (center left of map) is the Portuguese Bend Landslide, moving down and southwest toward the sea.

3) There are areas of positive and negative LOS change in the area of Long Beach (east part of map). These are due to the extraction of oil and injection of water in oilfields beneath the city and out in the harbor.

4) The block box at 33.76N, 118.36W is the reference pixel for this map, just north of the Portuguese Bend Landslide. 
</font>

# 3. Error Analysis (Separating Signal from Noise)


<font face="Calibri" size="3">Uncertainty of the ground displacement products derived from InSAR time-series, depends on the quality of the inversion of the stack of interferograms and the accuracy in separating the ground displacement from other components of the InSAR data. Therefore the definition of signal vs noise is different at the two main steps in mintpy:  

1) During the inversion: 
    At this step all systematic components of the interferometric phase (e.g., ground displacement, propagation delay, geometrical residuals caused by DEM or platform's orbit inaccuracy) are considered signal, while the interferometric phase decorrelation, phase unwrapping error and phase inconsistency are considered noise. 
    
2) After inversion: the ground displacement component of the time-serieses is signal, and everything else (including the propagation delay and geometrical residuals) are considered noise

Therefore we first discuss the possible sources of error during the inversion and the existing ways in MintPy to evaluate the quality of inversion and to improve the uncertainty of the inversion. Afterwards we explain the different components of the time-series and the different processing steps in MintPy to separate them from ground displacement signal.  
</font>

## 3.1 Quality of the Inversion

<font face="Calibri" size="3">The main sources of noise during the time-series inversion includes decorrelation, phase unwrapping error and the inconsistency of triplets of interferofgrams. Here we mainly focus on the decorrelation and unwrapping errors. We first show the existing quantities in MintPy to evaluate decorrelation and unwrapping errors and then discuss the existing ways in MintPy to reduce the decorrelation and unwrapping errors on the time-series inversion.</font>

### 3.1.1 Average Spatial Coherence

<font face="Calibri" size="3">Mintpy computes temporal average of spatial coherence of the entire stack as a potential ancillary measure to choose reliable pixels after time-series inversion. </font>

In [ ]:
view.main('avgSpatialCoh.h5 --noverbose'.split())

### 3.1.2 Temporal Coherence

<font face="Calibri" size="3">In addition to timeseries.h5 which contains the time-series dataset, invert_network produces other quantities, which contain metrics to evaluate the quality of the inversion including temporalCoherence.h5. Temporal coherence represents the consistency of the timeseries with the network of interferograms. 

Temporal coherence varies from 0 to 1. Pixels with values closer to 1 are considered reliable and pixels with values closer to zero are considered unreliable. For a dense network of interferograms, a threshold of 0.7 may be used (Yunjun et al, 2019). Here we are using a threshold of 0.5.</font>

In [ ]:
view.main('temporalCoherence.h5 --noverbose'.split())

<font face="Calibri" size="3">With both the spatial coherence and temporal coherence, we can see that the InSAR in the ports of Long Beach and Los Angeles have unstable phase, and the InSAR measurements there will be low quality.</font>

<div class="alert alert-danger">
<font size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>ASSIGNMENT #2</u>:  </font> Temporal Coherence:</b> <font color='rgba(200,0,0,0.2)'> -- [4 Points] </font> </font>
Temporal Coherence quantifies the consistency between the observed interferograms and the inverted phase time series. Areas where these are consistent show a coherence close to 1. Areas where the inverted time series doesn't match the observed InSAR phases well receive low coherence. Please name two reasons why temporal coherence may be low. 

</div>

<hr>
<div class="alert alert-danger">
<i><font color='rgba(200,0,0,0.2)'> Question 2.1 [2 Points]:</font></i> 

NAME YOUR <b>FIRST</b> POTENTIAL REASON FOR LOW TEMPORAL COHERENCE by adding text below:

</div>

<hr>
<div class="alert alert-danger">
<i><font color='rgba(200,0,0,0.2)'> Question 2.1 [2 Points]:</font></i> 

NAME YOUR <b>SECOND</b> POTENTIAL REASON FOR LOW TEMPORAL COHERENCE by adding text below:
</div>

## 3.2. Velocity Error Analysis

<font face="Calibri" size="3">The estimated velocity also comes with an expression of unecrtainty which is simply based on the goodness of fit while fitting a linear model to the time-series. This quantity is saved in "velocity.h5" under the velocityStd dataset. 

**Mintpy supports additional corrections in its processing not included in this demo:**
- Unwrapping error correction
- Tropospheric delay correction
- deramping
- Topographic residual correction
- Residual RMS for noise evaluation
- Changing the reference date
</font>

In [ ]:
scp_args = 'velocity.h5 velocityStd -v 0 0.2 --noverbose'
view.main(scp_args.split())

<font face="Calibri" size="3">Note that the plot above is the velocity error, not the velocity. The errors generally increase with distance from the reference point and also increase for points with elevations different from the reference point because of topographically correlated water vapor variations that are especially strong in this area.</font>

<div class="alert alert-danger">
<font size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>ASSIGNMENT #3</u>:  </font> Displacement Velocity Error Discussion:</b> <font color='rgba(200,0,0,0.2)'> -- [8 Points] </font> </font>
What are the main sources of error that results in the increase of velocity error with distance from the reference point? Name two likely sources of error below.

</div>

<hr>
<div class="alert alert-danger">
<i><font color='rgba(200,0,0,0.2)'> Question 3.1 [4 Points]:</font></i> 

NAME AND BRIEFLY DESCRIBE <b>FIRST</b> SOURCE OR ERROR AFFECTING DISPLACEMENT VELOCITY ACCURACY by adding text below:
</div>

<hr>
<div class="alert alert-danger">
<i><font color='rgba(200,0,0,0.2)'> Question 3.2 [4 Points]:</font></i> 

NAME AND BRIEFLY DESCRIBE <b>SECOND</b> SOURCE OR ERROR AFFECTING DISPLACEMENT VELOCITY ACCURACY by adding text below:
</div>

## 3.3. Phase Unwrapping Error Detection

<font face="Calibri" size="3">The interferometric phases are wrapped (modulo 2$\pi$) and integration of the wrapped phase, commonly called <b>phase unwrapping</b>, is required to obtain a field of relative phase with respect to a given pixel. The phase unwrapping algorithms add integer number of 2$\pi$ phase jumps to recover the unwrapped phase. Interferometric phase noise and discontinuities among different coherent regions may lead to wrong 2$\pi$ jumps added to the phase field known as unwrapping error. Unwrapping errors can bias the estimated time-series. </font>

<font face="Calibri" size="3">For an <b>interferogram triplet</b> ($\Delta\phi^{ij}$, $\Delta\phi^{jk}$ and $\Delta\phi^{ik}$), unwrapping errors will introduce an non-zero integer component $C_{int}^{ijk}$ in the closure phase $C^{ijk}$. Therefore, the number of interferogram triplets with non-zero integer ambiguity $T_{int}$ can be used to detect unwrapping errors:

$$C^{ijk}=\Delta\phi^{ij}+\Delta\phi^{jk}-\Delta\phi^{ik}$$
$$C_{int}^{ijk}=\frac{C^{ijk}-wrap(C^{ijk})}{2\pi}$$
$$T_{int}=\sum_{i=1}^T(C_{int}^{ijk}!=0)$$

where $warp$ is an operator to wrap the input number into $[-\pi, \pi)$; $T$ is the number of interferogram triplets. </font>

In [ ]:
!smallbaselineApp.py LASenDT71.txt --dostep quick_overview
plot_num_triplet_with_nonzero_integer_ambiguity('numTriNonzeroIntAmbiguity.h5', display=True, fig_size=[15, 5])

<div class="alert alert-success">
<font face="Calibri" size="3"><b>Take home messages from $T_{int}$ map and histogram</b>:

1. Areas with $T_{int}$ > 0 have unwrapping errors.
2. Areas share common positive $T_{int}$ value could be corrected.
3. Areas with widely-distributed $T_{int}$ value indicates random unwrapping errors, which are difficult to correct.</font>
</div>

## 3.4. Phase Unwrapping Error Correction

<font face="Calibri" size="3">MintPy provides three methods to possibly detect and correct the phase unwrapping errors. The first method is based on the phase closure of the triplets of the interferograms. The second methods is automating the traditional manual bridging method in which coherent components with the smallest distance from each other are assumed connected and therefore the a smooth phase variation across them are enforced. The third approach is a hybrid approach and simply uses the both approached mentioned before.

Note that to use the phase closure approach a dense network of interferograms should be available.

Below, we add an item to the MintPy configuration file to perform phase unwrapping using the ```bridging``` method:
</font>

In [ ]:
config_file = os.path.join(mint_dir, "LASenDT71.txt")
write_config_file(config_file, "mintpy.unwrapError.method = bridging")

<font face="Calibri" size="3">Now we can <b>perform phase unwrapping correction</b>:
</font>

In [ ]:
!smallbaselineApp.py LASenDT71.txt --dostep correct_unwrap_error

<font face="Calibri" size="3">After correcting for unwrapping errors, we need to <b>re-run the inversion</b> to see the impact of the unwrapping error correction.
</font>

In [ ]:
!smallbaselineApp.py LASenDT71.txt --dostep invert_network

<font face="Calibri" size="3">Now we can <b>re-plot the temporal coherence</b> to identify potential improvements through phase unwrapping correction (in this case, improvements are pretty small):
</font>

In [ ]:
view.main('temporalCoherence.h5 --noverbose'.split())

## 3.5. Residual DEM Error Correction 

<font face="Calibri" size="3">This step corrects the phase residual caused by the inaccuracy of the DEM used in InSAR processing (DEM error). To correct for DEM errors, we use its relationship with the perpendicular baseline time-series (Fattahi and Amelung, 2013, IEEE-TGRS).
</font>

In [ ]:
write_config_file(config_file, "mintpy.topographicResidual = yes")
!smallbaselineApp.py LASenDT71.txt --dostep correct_topography

<font face="Calibri" size="3">Low <b>let's view the calculated DEM error</b>:
</font>

In [ ]:
view.main('demErr.h5 --zero-mask -v -25 25 --noverbose'.split())

## 3.5. Re-Estimating Velocity After Error Correction

<font face="Calibri" size="3">Now we can <b>re-plot the deformation velocity as well as the temporal coherence</b> to identify potential improvements through phase unwrapping correction (in this case, improvements are pretty small):
</font>

In [ ]:
!smallbaselineApp.py LASenDT71.txt --dostep velocity
view.main('velocity.h5 velocity -v -1 1 --noverbose'.split())

# 4. Plotting a Landslide Motion Transect 

In [ ]:
scp_args = 'velocity.h5 --start-lalo 33.75 -118.38 --end-lalo 33.72 -118.3 '
plot_transection.main(scp_args.split())

!smallbaselineApp.py smallbaselineApp.cfg --dostep google_earth

<font face="Calibri" size="3">On this transect, the Portuguese Bend Landslide has an average velocity over the last five years that reaches more than 2.5 cm/year in the descending track radar LOS direction (note that the "zero" is about -5 so we have to subtract that first). By analyzing the velocity on the ascending track and combining the two LOS directions, then we could find out that the displacments are largely horizontal, westward (and southward that we can't see with InSAR).</font>

# 5. Validation - Comparing InSAR and GPS Velocities 

<font face="Calibri" size="3">MintPy's analysis is independent of GPS observations. This allows validating InSAR products with GPS data when they are available. For this purpose, we will download GPS data over the region of interest from Nevada Geodetic Laboratory at University of Nevada, Reno.

First. we write the University of Nevada, Reno GPS station holdings metadata to file ```GPS_stations.csv```.
</font>

In [ ]:
with asfn.work_dir(mint_dir):
    url = 'http://geodesy.unr.edu/NGLStationPages/DataHoldings.txt'
    response = urllib.request.urlopen(url, timeout=5)
    content = response.read()
    rows = content.decode('utf-8').splitlines()
    holdings_txt = Path('.')/'DataHoldings.txt'
    if holdings_txt.exists():
        holdings_txt.unlink()

with open(f'{mint_dir}/GPS_stations.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',', escapechar=',', quoting=csv.QUOTE_NONE)
    for row in rows:
        csv_writer.writerow([re.sub('\s+', ' ', row)])

<font face="Calibri" size="3">Now we can <b>build a list of GPS stations</b> within your area of interest and <b>select a station we want to use as reference</b>. We will set both InSAR and GPS velocities to zero at this location to be able to compare the two data sets directly.</font>

In [ ]:
# get the InSAR stack's corner coordinates
with h5py.File(f"{mint_dir}/inputs/geometryGeo.h5", 'r') as f:
    lon_west = float(f.attrs['LON_REF1'])
    lon_east = float(f.attrs['LON_REF2'])
    lat_south = float(f.attrs['LAT_REF1'])
    lat_north = float(f.attrs['LAT_REF3'])

# get the start and end dates of the timeseries
info = gdal.Info(f"{mint_dir}/timeseries_demErr.h5", format='json')
ts_start = info['metadata']['']['START_DATE']
ts_start = datetime.strptime(ts_start, '%Y%m%d')
ts_end = info['metadata']['']['END_DATE']
ts_end = datetime.strptime(ts_end, '%Y%m%d')

# find all stations that have data within the ts time range,
# are located within the AOI and at an unmasked pixel location
gps_stations = list()
with open(f'{mint_dir}/GPS_stations.csv', newline='') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in list(csv_reader)[1:]:
        begin_date = datetime.strptime(row[7], '%Y-%m-%d')
        mod_date = datetime.strptime(row[9], '%Y-%m-%d')
        lat = float(row[1])
        if float(row[2]) > 180:
            lon = float(row[2]) - 360
        else:
            lon = float(row[2])

        n = [lat, lon]
        a = [lat_north, lon_west]
        b = [lat_south, lon_west]
        c = [lat_south, lon_east]
        ab = np.subtract(a, b)
        an = np.subtract(a, n)
        bc = np.subtract(b, c)
        bn = np.subtract(b, n)

        in_aoi = 0 <= np.dot(ab, an) <= np.dot(ab, ab) and 0 <= np.dot(bc, bn) <= np.dot(bc, bc)
        in_date_range = ts_start >= begin_date and ts_end <= mod_date
        
        if in_aoi and in_date_range:
            vel_file = f'{mint_dir}/velocity.h5'
            atr = mintpy.utils.readfile.read_attribute(vel_file)
            coord = mintpy.utils.utils.coordinate(atr, lookup_file=f'{mint_dir}/inputs/geometryRadar.h5')
            y, x = coord.geo2radar(lat, lon)[:2]
            msk = mintpy.utils.readfile.read(f'{mint_dir}/maskTempCoh.h5')[0]
            box = (x, y, x+1, y+1)
            masked = not msk[y, x]
            if not masked:
                gps_stations.append(row[0].strip())
                
if len(gps_stations) > 0:
    gps_station = asfn.select_parameter(gps_stations)
    print("Select a GPS station")
    display(gps_station)
else:
    print("No GPS stations found.")

<font face="Calibri" size="3">Now we <b>visualize GPS stations as colored dots</b> on top of the InSAR map. Ideally, the GPS "color" matches the InSAR "color" in the background.</font>

In [ ]:
scp_args = f"{mint_dir}/velocity.h5 velocity --show-gps --ref-gps {gps_station.value} --gps-comp enu2los --gps-label --figsize 12 12 --noverbose"
with asfn.work_dir(mint_dir):
    mintpy.view.main(scp_args.split())

# 6. Visualizing Displacement Time Series for Selected Points

<font face="Calibri" size="3">Now we can <b>visualize displacement time series information</b> For selected points. In the code cell below, we are plotting the time series for pixel location ```x=104''' and ```y=186'''. The selected point is shown as a red triangle in the first figure below. Feel free to change the pixel location and visualize time series in different locations!</font>

In [ ]:
%matplotlib inline
scp_args = f"{mint_dir}/timeseries_demErr.h5 --yx 186 104 -d=inputs/geometryGeo.h5 --figsize 14 12 --noverbose "
with asfn.work_dir(mint_dir):
    mintpy.tsview.main(scp_args.split())

# 7. Reference Material

<font face="Calibri" size="3">

- Original Notebook withe detailed description by Yunjun and Fattahi at: https://nbviewer.jupyter.org/github/insarlab/MintPy-tutorial/blob/master/smallbaselineApp_aria.ipynb

- Mintpy reference: *Yunjun, Z., H. Fattahi, F. Amelung (2019), Small baseline InSAR time series analysis: unwrapping error correction and noise reduction, preprint doi:[10.31223/osf.io/9sz6m](https://eartharxiv.org/9sz6m/).*

- University of Miami online time-series viewer: https://insarmaps.miami.edu/

- Mintpy Github repository: https://github.com/insarlab/MintPy

- ARIA-tools Github Repository: https://github.com/aria-tools/ARIA-tools
</font>

# 8. Change Log

<font face="Calibri" size="2"> <i>LosAngeles_time_series.ipynb - Version 1.2.1 - March 2022
<br>
    <b>Version Changes</b>
    <ul>
        <li>url_widget</li>
        <li>added more visualization functionality. Also added DEM error correction and Phase Unwrapping Error Correction.</li>
    </ul></i>
</font>